In [1]:
!pip install ultralytics
import torch
import cv2
import numpy as np
import pandas as pd
import math
import os
from ultralytics import YOLO

In [2]:
yolo_model = YOLO('yolo11n-pose.pt')

In [3]:
'''
import cv2
import torch
from ultralytics import YOLO

# טוען את המודל
yolo_model = YOLO('yolo11n-pose.pt')

# פותח את הסרטון
cap = cv2.VideoCapture('path_to_video.mp4')  # החלף בנתיב של הסרטון שלך

# בודק אם הסרטון נטען בהצלחה
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# קובע את קצב הפריימים
fps = cap.get(cv2.CAP_PROP_FPS)

# יוצא שם חדש לסרטון הפלט
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # פורמט לוידיאו MP4
out = cv2.VideoWriter(output_path, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# לולאת קריאת פריימים מהסרטון
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # מבצע זיהוי אובייקטים על כל פריים
    results = yolo_model(frame)  # הפעלת המודל על הפריים
    
    # results[0] מכילה את התוצאות מהמודל, ומכילה אובייקט מסוג 'Results'
    result_frame = results[0].plot()  # מצייר את התוצאות עם bounding boxes
    
    # שומר את הפריים המעובד בסרטון הפלט
    out.write(result_frame)

    # מציג את הסרטון בזמן אמת (אם תרצה)
    cv2.imshow('YOLO Detection', result_frame)

    # אם תלחץ על 'q', הלולאה תעצור
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# סוגר את הסרטון
cap.release()
out.release()
cv2.destroyAllWindows()

'''

'\nimport cv2\nimport torch\nfrom ultralytics import YOLO\n\n# טוען את המודל\nyolo_model = YOLO(\'yolo11n-pose.pt\')\n\n# פותח את הסרטון\ncap = cv2.VideoCapture(\'path_to_video.mp4\')  # החלף בנתיב של הסרטון שלך\n\n# בודק אם הסרטון נטען בהצלחה\nif not cap.isOpened():\n    print("Error: Could not open video.")\n    exit()\n\n# קובע את קצב הפריימים\nfps = cap.get(cv2.CAP_PROP_FPS)\n\n# יוצא שם חדש לסרטון הפלט\noutput_path = \'output_video.mp4\'\nfourcc = cv2.VideoWriter_fourcc(*\'mp4v\')  # פורמט לוידיאו MP4\nout = cv2.VideoWriter(output_path, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))\n\n# לולאת קריאת פריימים מהסרטון\nwhile cap.isOpened():\n    ret, frame = cap.read()\n    if not ret:\n        break\n    \n    # מבצע זיהוי אובייקטים על כל פריים\n    results = yolo_model(frame)  # הפעלת המודל על הפריים\n    \n    # results[0] מכילה את התוצאות מהמודל, ומכילה אובייקט מסוג \'Results\'\n    result_frame = results[0].plot()  # מצייר את התוצאות עם bounding boxes\n    \n    # שומר את הפרי

In [4]:
import cv2
import pandas as pd
from ultralytics import YOLO

# טוען את המודל
yolo_model = YOLO('yolo11n-pose.pt')

# פותח את הסרטון
cap = cv2.VideoCapture('\\Users\\user\\Desktop\\MarchPocFinalProject\\final_videos\\video_1.mp4')  # החלף בנתיב של הסרטון שלך

# בודק אם הסרטון נטען בהצלחה
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# רשימת איברים (על פי המודל YOLOv5 Pose)
keypoint_names = [
    "Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear", 
    "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow", 
    "Left Wrist", "Right Wrist", "Left Hip", "Right Hip", 
    "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"
]

# יצירת רשימה שתכיל את כל הנתונים שברצוננו לשמור
data = []

# לולאת קריאת פריימים מהסרטון
frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # מבצע זיהוי אובייקטים על כל פריים
    results = yolo_model(frame)  # הפעלת המודל על הפריים
    
    # results[0] מכילה את התוצאות מהמודל, ומכילה אובייקט מסוג 'Results'
    for result in results:
        keypoints = result.keypoints.data.cpu().numpy()  # נתוני נקודות המפתח
        
        # עבור כל מטריצה של keypoints
        for kp_set in keypoints:
            # עבור כל נקודת מפתח (שורה במטריצה)
            for i, kp in enumerate(kp_set):
                x, y, visibility = kp  # קואורדינטות ו-visibility (אם קיים)
                if visibility > 0.5:  # שומר רק נקודות עם visibility גבוה
                    # הוספת הנתונים לרשימה
                    data.append([frame_idx, keypoint_names[i], x, y, visibility])

    frame_idx += 1  # הגדלת מספר הפריים

# סגירת הסרטון
cap.release()

# יצירת DataFrame עם הנתונים שנאספו
df = pd.DataFrame(data, columns=["Frame", "Keypoint", "X", "Y", "Visibility"])

# הצגת ה־DataFrame
print(df)

# ייצוא לקובץ CSV
df.to_csv('pose_keypoints.csv', index=False)



0: 640x384 1 person, 62.2ms
Speed: 4.5ms preprocess, 62.2ms inference, 92.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 31.2ms
Speed: 1.7ms preprocess, 31.2ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 17.4ms
Speed: 1.3ms preprocess, 17.4ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 18.2ms
Speed: 1.0ms preprocess, 18.2ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 20.5ms
Speed: 1.3ms preprocess, 20.5ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 18.1ms
Speed: 1.1ms preprocess, 18.1ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 23.3ms
Speed: 1.6ms preprocess, 23.3ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 21.2ms
Speed: 1.3ms preprocess, 21.2ms inference, 2.3ms postprocess per image at shape (1, 3, 6

In [5]:
'''
import os
import pandas as pd
from ultralytics import YOLO
import cv2

# טוען את המודל
yolo_model = YOLO('yolo11n-pose.pt')

# יצירת רשימה להכיל את כל הנתונים
data = []

# הגדרת הנתיב לסרטונים
video_directory = r"C:\\Users\\user\\Desktop\\MarchPocFinalProject\\final_videos"

# שמות הנקודות המייצגות איברים
keypoint_names = [
    "Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear", 
    "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow", 
    "Left Wrist", "Right Wrist", "Left Hip", "Right Hip", 
    "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"
]

# עבור כל סרטון בנתיב
for video_num in range(1, 45):  # הסרטונים נקראים video_1.mp4 עד video_44.mp4
    video_path = os.path.join(video_directory, f"video_{video_num}.mp4")
    
    # קריאת הסרטון
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"לא ניתן לפתוח את הסרטון {video_path}")
        continue

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        
        # אם הסרטון נגמר, נעצור את הלולאה
        if not ret:
            break
        
        # הפעלת המודל על הפריים
        results = yolo_model(frame)
        
        for result in results:
            keypoints = result.keypoints.data  # נקודות מפתח (קואורדינטות ו-visibility)

            # אם אין נקודות מפתח (המערך ריק), נעבור לפריים הבא
            if len(keypoints) == 0:
                continue

            # עבור כל נקודת מפתח
            for i, kp in enumerate(keypoints):
                x, y, visibility = kp[0], kp[1], kp[2]  # קואורדינטות ו-visibility
                
                # הוספת שורה לנתונים
                data.append({
                    'Frame': frame_idx,
                    'Video': f'video_{video_num}.mp4',
                    'Keypoint': keypoint_names[i],
                    'X': x,
                    'Y': y,
                    'Visibility': visibility
                })
        
        frame_idx += 1
    
    cap.release()

# יצירת DataFrame
df = pd.DataFrame(data)

# שמירת הנתונים לקובץ CSV
df.to_csv('yolo_motion_dataset.csv', index=False)

print("הנתונים יוצאו בהצלחה ל-csv בשם yolo_motion_dataset.csv")
'''

'\nimport os\nimport pandas as pd\nfrom ultralytics import YOLO\nimport cv2\n\n# טוען את המודל\nyolo_model = YOLO(\'yolo11n-pose.pt\')\n\n# יצירת רשימה להכיל את כל הנתונים\ndata = []\n\n# הגדרת הנתיב לסרטונים\nvideo_directory = r"C:\\Users\\user\\Desktop\\MarchPocFinalProject\\final_videos"\n\n# שמות הנקודות המייצגות איברים\nkeypoint_names = [\n    "Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear", \n    "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow", \n    "Left Wrist", "Right Wrist", "Left Hip", "Right Hip", \n    "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"\n]\n\n# עבור כל סרטון בנתיב\nfor video_num in range(1, 45):  # הסרטונים נקראים video_1.mp4 עד video_44.mp4\n    video_path = os.path.join(video_directory, f"video_{video_num}.mp4")\n    \n    # קריאת הסרטון\n    cap = cv2.VideoCapture(video_path)\n    \n    if not cap.isOpened():\n        print(f"לא ניתן לפתוח את הסרטון {video_path}")\n        continue\n\n    frame_idx = 0\n    while True:\n      

In [6]:
import contextlib
import os
import sys

import logging

# משתיק את כל הלוגרים
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger("ultralytics").setLevel(logging.ERROR)
logging.getLogger("yolo").setLevel(logging.ERROR)
logging.getLogger("torch").setLevel(logging.ERROR)


@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr



# רשימת איברים (על פי המודל YOLO Pose)
keypoint_names = [
    "Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear", 
    "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow", 
    "Left Wrist", "Right Wrist", "Left Hip", "Right Hip", 
    "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"
]
yolo_model = YOLO("yolo11n-pose.pt", verbose=False) 

def process_single_video(video_num: int) -> pd.DataFrame:
    """
    פונקציה שמבצעת ניתוח של סרטון לפי מספרו, מחזירה DataFrame עם נקודות מפתח ופריים.
    """
    # יצירת נתיב הסרטון לפי המספר
    video_path = f'\\Users\\user\\Desktop\\MarchPocFinalProject\\final_videos\\video_{video_num}.mp4'
    print(f"Trying to open video: {video_path}")
    
    # ניסיון לפתוח את הסרטון
    cap = cv2.VideoCapture(video_path)

    # בדיקה אם נפתח בהצלחה
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return pd.DataFrame()  # מחזיר DataFrame ריק אם לא נפתח

    data = []
    frame_idx = 0

    # מעבר על כל פריים
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        with suppress_output():
            results = yolo_model(frame)

        for result in results:
            keypoints = result.keypoints.data.cpu().numpy()

            for kp_set in keypoints:
                for i, kp in enumerate(kp_set):
                    x, y, visibility = kp
                    if visibility > 0.5:
                        data.append([
                            frame_idx, keypoint_names[i], x, y, visibility
                        ])
        
        frame_idx += 1

    cap.release()

    df = pd.DataFrame(data, columns=["Frame", "Keypoint", "X", "Y", "Visibility"])
    return df



In [7]:
import pandas as pd

# רשימה לאגירת כל ה־DataFrame-ים
all_dfs = []

# לולאה על כל הסרטונים מ-1 עד 44
for video_num in range(1, 45):
    print(f"Processing video number {video_num}...")
    df = process_single_video(video_num)  # קריאה לפונקציה שלך
    
    if not df.empty:  # רק אם יש נתונים
        df["Video Number"] = video_num  # הוספת מספר הסרטון
        all_dfs.append(df)  # הוספה לרשימת כל הנתונים

# איחוד כל הנתונים ל־DataFrame אחד
full_df = pd.concat(all_dfs, ignore_index=True)

# שמירת הנתונים כ־CSV
full_df.to_csv("yolo_motion_dataset.csv", index=False)

print("Finished processing all videos. Data saved to yolo_motion_dataset.csv.")


Processing video number 1...
Trying to open video: \Users\user\Desktop\MarchPocFinalProject\final_videos\video_1.mp4
Processing video number 2...
Trying to open video: \Users\user\Desktop\MarchPocFinalProject\final_videos\video_2.mp4
Processing video number 3...
Trying to open video: \Users\user\Desktop\MarchPocFinalProject\final_videos\video_3.mp4
Processing video number 4...
Trying to open video: \Users\user\Desktop\MarchPocFinalProject\final_videos\video_4.mp4
Processing video number 5...
Trying to open video: \Users\user\Desktop\MarchPocFinalProject\final_videos\video_5.mp4
Processing video number 6...
Trying to open video: \Users\user\Desktop\MarchPocFinalProject\final_videos\video_6.mp4
Processing video number 7...
Trying to open video: \Users\user\Desktop\MarchPocFinalProject\final_videos\video_7.mp4
Processing video number 8...
Trying to open video: \Users\user\Desktop\MarchPocFinalProject\final_videos\video_8.mp4
Processing video number 9...
Trying to open video: \Users\user\D